In [1]:
import numpy as np
import pickle
from sklearn.linear_model import ElasticNetCV, LogisticRegressionCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import norm
from data_generation import m_0, g_0, get_data
from dml_algorithm import mm_ate, dml_ate

In [2]:
rng = np.random.default_rng(seed=42)

In [12]:
N = 2000
y_data, d_data, x_data = get_data(N, rng)
poly_features = PolynomialFeatures(degree=2, include_bias=False)
x_quad_data = poly_features.fit_transform(x_data)
y_train, y_test, d_train, d_test, x_train, x_test, x_quad_train, x_quad_test = train_test_split(y_data, d_data, x_data, x_quad_data, test_size=0.2, random_state=42)
x_quad_data.shape

(2000, 65)

In [14]:
l1_ratio = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
alphas = [0.01, 0.1, 1, 10]
model_g0, model_g1 = ElasticNetCV(l1_ratio=l1_ratio, n_alphas=10, max_iter=5000, n_jobs=-1), ElasticNetCV(l1_ratio=l1_ratio, n_alphas=10, max_iter=5000, n_jobs=-1)
model_m = LogisticRegressionCV(Cs=10, 
                               l1_ratios=[0, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 1], 
                               penalty='elasticnet',
                               solver='saga',
                               max_iter=1000,
                               random_state=42,
                               scoring='neg_brier_score',
                               n_jobs=-1
                              )

In [15]:
%%time
model_m.fit(x_train, d_train)
print(model_m.C_, model_m.l1_ratio_)
print(model_m.predict_proba(x_test)[:20,1])
print(m_0(x_test[:20]))

[0.35938137] [0]
[0.57132776 0.70567737 0.4303399  0.686172   0.27521405 0.70580836
 0.33302485 0.19530819 0.19380231 0.85319384 0.50408401 0.24799784
 0.7783664  0.57683768 0.1487862  0.08795634 0.38613909 0.28276003
 0.16569189 0.73896637]
[0.53530715 0.96431205 0.31125886 0.65458118 0.53836062 0.66536453
 0.33213458 0.1322376  0.17076338 0.76309375 0.41828115 0.23771557
 0.8244302  0.64180668 0.09459765 0.16131046 0.27140645 0.18575402
 0.16098517 0.67452492]
CPU times: total: 3.12 s
Wall time: 1.23 s


In [18]:
%%time
model_m = LogisticRegressionCV(Cs=5, 
                               #l1_ratios=[0, 0.5, 0.9, 1], 
                               penalty='l1',
                               solver='liblinear',
                               max_iter=10000,
                               random_state=42,
                               scoring='neg_brier_score',
                               n_jobs=-1
                              )

model_m.fit(x_quad_train, d_train)
print(model_m.C_, model_m.l1_ratio_)
print(model_m.predict_proba(x_quad_test)[:20,1])
print(m_0(x_test[:20]))

[1.] [None]
[0.53407733 0.94079336 0.37795981 0.53399177 0.28480214 0.51341574
 0.37732521 0.15515152 0.13823188 0.89917877 0.51663389 0.21402093
 0.79567239 0.66727549 0.09599924 0.06825794 0.26883812 0.12538092
 0.1438994  0.67705404]
[0.53530715 0.96431205 0.31125886 0.65458118 0.53836062 0.66536453
 0.33213458 0.1322376  0.17076338 0.76309375 0.41828115 0.23771557
 0.8244302  0.64180668 0.09459765 0.16131046 0.27140645 0.18575402
 0.16098517 0.67452492]
CPU times: total: 281 ms
Wall time: 4.24 s


In [19]:
model_m.coef_

array([[-0.91074815, -0.34047614, -0.47341033, -0.37899903,  0.        ,
         0.54439084,  0.94433671,  0.77377931,  1.28622661,  0.        ,
         0.03793427, -0.06048389, -0.08385394,  0.00764856, -0.0246025 ,
        -0.05041171, -0.0579867 ,  0.09066764, -0.40685654, -0.16276652,
        -0.18413837, -0.35995399,  0.07573861,  0.08196914, -0.14789234,
        -0.20848969,  0.00501321, -0.4061843 ,  0.        , -0.10249395,
         0.        ,  0.08134478, -0.05367085, -0.04064879,  0.02916761,
         0.        ,  0.14451927, -0.04380632,  0.04360667,  0.107183  ,
        -0.01330358,  0.05429709, -0.15188563,  0.        , -0.03206743,
        -0.07323135, -0.01710169,  0.16793046, -0.23825873,  0.34772434,
        -0.03231787,  0.03126953,  0.        , -0.04418689,  0.04087321,
         0.05710834,  0.08898668,  0.03709847,  0.        ,  0.4142498 ,
         0.00527233,  0.18311688,  0.        , -0.23492561,  0.82477821]])

In logistic regression, we do not include second-order interaction terms since convergence issues and not-converged model was still very bad and model without interaction terms was much better!

In [20]:
"""
%%time
model_g0.fit(x_train[d_train==0], y_train[d_train==0])
print(model_g0.alpha_, model_g0.l1_ratio_)
print(model_g0.predict(x_test[:20]))
print(g_0(0, x_test[:20]))
"""

0.002023169961802633 1.0
[ 9.11913773  1.86267306  3.38823435  3.92009905  3.71135644  8.01888979
  6.47477631  3.35071711 -2.48495728  3.26023665  4.01562794  2.40326794
  5.08584477  7.09953179  0.24808764 10.10317209  4.97710924  6.61279589
  4.84095057  8.4415757 ]
[ 9.67089239  2.52296456  2.95237173  3.3036634   3.46981129  7.25168064
  5.95556265  4.4787683  -2.17647036  2.9592268   3.37120465  2.49881907
  6.21870476  6.53252436 -0.14063823  9.19354424  4.95250245  6.55058744
  4.15508761  7.76348105]
CPU times: total: 391 ms
Wall time: 397 ms


In [20]:
%%time
model_g0.fit(x_quad_train[d_train==0], y_train[d_train==0])
print(model_g0.alpha_, model_g0.l1_ratio_)
print(model_g0.predict(x_quad_test[:20]))
print(g_0(0, x_test[:20]))

0.009998977262763385 1.0
[ 5.37263919  7.82256437  5.79846071  0.31651282  1.63767     6.26884193
  7.07877348  5.91309718  6.43477437  5.82649595  2.72411738  5.16889343
  5.52335172  4.75242544  3.06413003 10.05380554  5.26490172  6.06307021
  1.7858347   6.38418074]
[ 5.86015472  8.44071351  5.99188868  0.20648479  1.19112596  5.91414236
  7.11103775  6.43084221  6.08326961  6.09704997  3.23334733  5.5079997
  5.3976778   4.45940225  3.3907018  10.79692326  5.50558326  5.86234776
  1.38708098  6.33837782]
CPU times: total: 594 ms
Wall time: 345 ms


In [21]:
model_g0.coef_

array([ 0.50639481,  1.54679122,  1.67077498,  2.61919867,  0.62748503,
        0.        , -0.32600152,  0.        , -0.20664623, -0.        ,
        0.05427592,  0.17422959,  0.16538051,  0.11888941,  0.12685507,
        0.        , -0.07972751, -0.02960964, -0.03736361,  0.0161755 ,
        0.04834936,  0.00267701,  0.11255661,  0.11482012,  0.10334559,
        0.02709832, -0.05593316,  0.        , -0.        ,  0.        ,
        0.09777721,  0.06194293,  0.08544696,  0.        , -0.01462962,
       -0.        , -0.        ,  0.02192548,  0.02242899, -0.01625273,
       -0.02596337, -0.        ,  0.        ,  0.0119906 ,  0.0577651 ,
        0.08732983,  0.00813324,  0.        , -0.        ,  0.        ,
       -0.03487082,  0.0321096 ,  0.08227512,  0.06901976,  0.08109427,
        0.47852579,  0.01678953, -1.67344701, -0.        , -0.02745161,
        0.        ,  0.        , -0.06703705, -0.65608533,  0.        ])

In [22]:
"""
%%time
model_g1.fit(x_train[d_train==1], y_train[d_train==1])
print(model_g1.alpha_, model_g1.l1_ratio_)
print(model_g1.predict(x_test[:20]))
print(g_0(1, x_test[:20]))
"""

0.0016604646032669184 0.95
[ 8.53598317  4.02933957  5.30445082  5.87899583  3.18154968  9.43416427
  5.66363481  5.57276874 -1.60926805  2.97616642  3.14048025  4.77194485
  5.46062975  7.30029159  1.50686742  9.49802442  6.33179126  6.76632479
  3.56087634  7.92841459]
[ 8.44765297  4.82355923  4.62288453  5.72266669  2.47828193  8.59848846
  4.87169894  7.07646335 -1.75255434  2.34590245  2.36804108  5.45466763
  6.86649874  6.80355745  1.014024    8.8125616   6.12793833  6.2625267
  2.89036964  6.58935663]
CPU times: total: 250 ms
Wall time: 198 ms


In [22]:
%%time
model_g1.fit(x_quad_train[d_train==1], y_train[d_train==1])
print(model_g1.alpha_, model_g1.l1_ratio_)
print(model_g1.predict(x_quad_test[:20]))
print(g_0(1, x_test[:20]))

0.013232760541359396 1.0
[ 6.65315796  8.67523475  5.98742194  1.3064866   3.17139834  5.4648934
  6.32332282  6.77944959  5.35386695  8.43901092  4.97859624  6.34858337
  5.51759695  3.72356525  5.88319003 10.59649137  5.20167916  7.10001918
  3.48388497  6.6870507 ]
[ 6.75976037  9.47459189  6.14234082  1.46327632  2.99225022  5.48315741
  6.07698098  7.23117959  5.06054911  8.44959794  5.10641576  6.70174088
  5.14651884  3.92870919  6.04695953 11.18807021  5.39208888  6.76624743
  2.92245927  6.44924969]
CPU times: total: 438 ms
Wall time: 324 ms


In [23]:
model_g1.coef_

array([ 9.58333533e-01,  1.89061904e+00,  2.00546204e+00,  2.89218099e+00,
        1.90820433e+00,  1.84543562e-01, -3.96427732e-01,  0.00000000e+00,
       -2.83817128e-01, -4.62658122e-02, -0.00000000e+00,  5.75617383e-02,
        4.02634390e-02,  5.16270521e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -3.35385230e-02,  0.00000000e+00,  9.06858534e-03,
        2.63702561e-02, -0.00000000e+00,  4.49356251e-02,  0.00000000e+00,
        4.50415441e-02,  5.60973550e-02, -1.74166473e-02, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  6.33015229e-02, -0.00000000e+00,
       -0.00000000e+00,  1.51251672e-02, -2.49383592e-03,  6.94430107e-01,
        0.00000000e+00,  2.58722194e-02,  3.84965546e-02, -0.00000000e+00,
        0.00000000e+00,  8.20683649e-02, -0.00000000e+00,  0.00000000e+00,
        4.09997607e-02, -4.43060301e-02, -6.74031900e-02, -2.26723004e-02,
        0.00000000e+00, -0.00000000e+00, -5.19943079e-02,  0.00000000e+00,
       -4.32651954e-02,  

In [25]:
def dml_ate(y_data, d_data, x_data, x_quad_data, model_g, model_m, K=5, classical=True, inference=True, alpha=0.05):
    # Generate random partition of data for cross-fitting
    N = len(y_data)
    skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

    # Compute respective ML estimators and thereupon auxiliary estimators
    theta_0_check_list = []
    if classical:
        reg_check_list, ipw_check_list = [], []
    if inference:
        scores_list = []
    
    for (train_indices, eval_indices) in skf.split(X=x_data, y=d_data):
        y_train, d_train, x_train, x_quad_train = y_data[train_indices], d_data[train_indices], x_data[train_indices], x_quad_data[train_indices] 
        y_eval, d_eval, x_eval, x_quad_eval = y_data[eval_indices], d_data[eval_indices], x_data[eval_indices], x_quad_data[eval_indices] 

        # Estimate outcome regression functions g_0(d)
        g_0_hat = []
        for d in [0, 1]:
            model_g[d].fit(X=x_quad_train[d_train==d], y=y_train[d_train==d])
            g_0_hat.append(model_g[d].predict(x_quad_eval))

        # Estimate propensity score m_0
        model_m.fit(X=x_train, y=d_train)
        m_0_hat = model_m.predict_proba(x_eval)[:,1]
            
        # Compute auxiliary estimator
        scores = g_0_hat[1] - g_0_hat[0] + d_eval*(y_eval-g_0_hat[1])/m_0_hat - (1-d_eval)*(y_eval-g_0_hat[0])/(1-m_0_hat)
        theta_0_check_list.append(np.mean(scores))

        # For variance estimation
        if inference:
            scores_list.append(scores)

        # For regression & IPW estimators
        if classical:
            reg_check_list.append(np.mean(g_0_hat[1] - g_0_hat[0])) 
            ipw_check_list.append(np.mean(d_eval*y_eval/m_0_hat - (1-d_eval)*y_eval/(1-m_0_hat)))     

    # Compute final estimator
    theta_0_hat = np.mean(theta_0_check_list)
    if classical:
        reg_hat, ipw_hat = np.mean(reg_check_list), np.mean(ipw_check_list)

    # Inference: estimate variance and construct confidence interval
    if inference:
        sigma_hat = np.sqrt(np.mean((np.array(scores_list)-theta_0_hat)**2))
        quantile = norm.ppf(1-alpha/2)
        CI = np.array([theta_0_hat-quantile*sigma_hat/np.sqrt(N), theta_0_hat+quantile*sigma_hat/np.sqrt(N)])

    # Return results
    if classical:
        if inference:
            return np.array([theta_0_hat, reg_hat, ipw_hat]), sigma_hat, CI
        else:
            return np.array([theta_0_hat, reg_hat, ipw_hat])
    else:
        if inference:
            return theta_0_hat, sigma_hat, CI
        else:
            return theta_0_hat

In [53]:
%%time
model_g = [model_g0, model_g1]
dml_ate(y_data, d_data, x_data, x_quad_data, model_g, model_m)

CPU times: total: 8.69 s
Wall time: 3.6 s


(array([0.62729263, 0.56263539, 0.25905684]),
 2.760932977028119,
 array([0.45617138, 0.79841388]))